## A/B Testing eCommerce Site with BigQuery

This report explores how we can analyse and visualise the results from an A/B test. If you want to run this notebook, the install instructions are kept [in the Readme.md](/Readme.md).

### Introduction

A/B testing refers to an experiment technique to determine whether a new design brings improvement, according to a chosen metric. The idea is to compare an existing version of a feature on a website (A) with a new one (B), by randomly splitting traffic and comparing metrics on each of the splits. A feature can be a whole web page or simply the colour of a button.

In this report we will learn how to pull in the data from BigQuery and analyse results of an A/B test giving valuable insights for an online eCommerce platform, wishing to test their landing pages and drive higher purchase rates.

### Context

A company selling hotel rooms across Spain wants to see what landing page is better to direct potential customers. Does page A or page B give us a better purchase rate?

**Page A** would land the customer on the homepage of the site, with a main search bar, a view of all the regions in Spain and a few best selling hotel options. 
**Page B** would land the customer on a city / region page related to the search query of the customer. For example “hotels in Mallorca”.

In order to determine which page will give the better purchase rate, we will split the incoming traffic 50:50 to both pages.

Our evaluation metrics will include:
- **Average time per session**
- **Purchase rate**

### Evaluation

In order to statistically evaluate these metrics we will be using a standard t-test for average time per session. The results will be evaluated at 90%, 95% and 99% confidence levels.
Purchase rate, since a binary output (either made a purchase (1) or did not (0)) we will use the z-test, which gives a similar result to the chi-squared test.

First let's import the libraries we will require:

In [1]:
import pandas as pd
import json
import numpy as np
import os

import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
from cufflinks import tools

from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

from google.cloud import bigquery

### Create Global Variables For Data Access

If you haven't already, check out the Readme.md file on this repository for instructions on how to creeate a project in the Google API console to enable the API, and create the key file, which you should save to your machine.

Replace the path_to_json value below with your own.

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "path_to_json"

### Fetching Our Data

Below we are initializing an BigQuery API with our credentials created above, defining our queries to a BigQuery table and returning its response.

In [9]:
client = bigquery.Client()

query_job = client.query("""
    SELECT
      *
    FROM `project_name.database_name.table_name`
       """)

results = query_job.result() 

Next we'll parse the results through a row iterator and save the data in a pandas dataframe.

In [42]:
df = pd.DataFrame(columns=['Session_Id','Purchase','Session_Time','Landing_Page'])
for row in results:
    df = df.append({'Session_Id':row['Session_Id'],'Purchase':row['Purchase'],
               'Session_Time':row['Session_Time'],'Landing_Page':row['Landing_Page']},ignore_index=True)

### Caching Our Data

Let's save everything to cache file so we dont need to download all the time. This is also handy to save data for APIs that have historical limits.

In [426]:
with open('cache/ab_test.json', 'w') as f:
    f.write(df.to_json())

In [427]:
df = pd.read_json('cache/ab_test.json')

In [407]:
df = pd.read_csv('ab_test_data_v2.csv')

In [428]:
df.head()

,Landing Page,Purchase,Purchase Amount,Session Id,Session Time,Timestamp,YearWeek
0,B,1,18.353593,10001,3.09,2020-01-22 12:06:00,2020-w03
1,B,0,0.000000,10003,4.60,2020-01-22 12:38:00,2020-w03
10,B,1,41.201243,10021,5.59,2020-01-22 17:26:00,2020-w03
100,B,0,0.000000,10201,2.70,2020-01-24 17:26:00,2020-w03
1000,B,0,0.000000,12054,4.76,2020-02-14 07:34:00,2020-w06


### Preparing our Data

Let's prepare the data into arrays to be used later in the analysis.

In [429]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['YearWeek'] = df['Timestamp'].dt.strftime('%Y-w%U')

In [430]:
session_time_A = df.loc[df['Landing Page'] == 'A','Session Time'].values
session_time_B = df.loc[df['Landing Page'] == 'B','Session Time'].values

purchases_A = df.loc[df['Landing Page'] == 'A','Purchase'].values
purchases_B = df.loc[df['Landing Page'] == 'B','Purchase'].values

### What Are The Average Values per Landing Page?

Let's visualize the average purchase rate and session time for each landing page.

In [431]:
df.groupby('Landing Page').agg({'Session Time':'mean','Purchase':'mean','Purchase Amount':'mean'})

,Session Time,Purchase,Purchase Amount
Landing Page,,,
A,4.010490,0.072524,2.183386
B,3.898219,0.094994,2.826410


In [432]:

fig = df.groupby('Landing Page').agg({'Session Time':'mean','Purchase':'mean','Purchase Amount':'sum'})\
    .iplot(asFigure=True, kind='bar', 
  subplots=True, subplot_titles=True, 
        legend=False)

fig.layout.height = 600
fig.layout.width = 800
fig.layout.template = 'plotly'

fig.layout.xaxis1.title='Landing Page'
fig.layout.xaxis2.title='Landing Page'
fig.layout.xaxis3.title='Landing Page'
fig.layout.yaxis1.title='Session Time (s)'
fig.layout.yaxis2.title='Purchase Rate (%)'
fig.layout.yaxis3.title='Purchase Amount ($)'

fig.layout.yaxis2.tickformat = ',.0%'
fig.layout.yaxis1.ticksuffix = 's'

fig.show()

Visually, it appears that landing page B has a higher purchase, while landing page A may have a slightly longer session time.

Let's see if these insights hold true when put through statistical analysis.

### What Is The Purchase Amount Over Time per Landing Page?

In [433]:
temp = df.groupby(['YearWeek','Landing Page']).agg({'Purchase':'sum','Session Time':'mean',
                                                   'Purchase Amount':'sum',
                                                   'Session Id':'count'}).reset_index()
temp.rename(columns={'Session Id':'Count'},inplace=True)

temp = temp.pivot(index='YearWeek',columns='Landing Page',values='Purchase Amount').reset_index()

layout = cf.Layout(
    height = 600,
    width = 800,
    yaxis = dict(
    title = 'Purchase Amount ($)'),
    xaxis = dict(
    title = 'Week'),
    title = 'Total Weekly Purchase Amount Over Time'
)

fig = temp.set_index('YearWeek')[['A','B']].\
    iplot(kind='scatter',
          fill=True,
          width=2,
          asFigure=True,
          layout = layout)

fig.show()

### Average Session Time Test Analysis

**H0:** Landing pages A and B show no difference in average purchase rate

In order to evaluate the session time result, we will use the ttest from the scipy package.

In [434]:
stat, p = ttest_ind(session_time_A, session_time_B)
print('Statistics=%.3f, p=%.3f' % (stat, p))

Statistics=7.158, p=0.000


Since the p value is 0 we are able to reject the null hypothesis at 99% confidence level.

In [435]:
print('Landing page A has an average session time higher than page B by:')
print(round((session_time_A.mean()/session_time_B.mean()-1)*100,2),'%')

Landing page A has an average session time higher than page B by:
2.88 %


### Purchase Rate Test Analysis

**H0:** Landing pages A and B show no difference in average conversion rate

In order to evaluate the conversion rate result, we will use the proportions ztest from the statsmodels package, a similar method to the chi-squared test.

Each variable explained:
- Count - the number of successes (conversions) in each sample.
- Nobs - the sample size of each sample.

In [436]:
count = np.array([purchases_A.sum(), purchases_B.sum()])
nobs = np.array([len(purchases_A), len(purchases_B)])
stat, p = proportions_ztest(count, nobs)

print('Statistics=%.3f, p=%.3f' % (stat,p))

Statistics=-4.094, p=0.000


The p value is 0, which means we can reject the null hypothesis with 99% confidence. We can conclude that there is a statistically significant difference between the convesion rates of landing page A and B, with landing page B having a higher rate.

In [437]:
print('Landing page B has an average purchase rate higher than page A by:')
print(round((purchases_B.sum()/len(purchases_B)-purchases_A.sum()/len(purchases_A))*100,2),'% points')

Landing page B has an average purchase rate higher than page A by:
2.25 % points


### Final Evaluation

- The overall results show that while session time is higher on landing page A, the customers are more likely to buy on page B.
- This may be because, while on page A, they are able to browse the homepage and spend more time looking for the page / hotel they are interested in, page B takes them deeper into the funnel, directly to the area they're looking for. This makes page B more relevant and likely to buy.